In [2]:
from __future__ import division
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import torch.optim as optim
from torch import nn
import matplotlib.pyplot as plt
from utils import generate_dataset, get_normalized_adj, get_Laplace, calculate_random_walk_matrix,nb_zeroinflated_nll_loss,nb_zeroinflated_draw,nb_MDN_nll_loss
from model import *
import random,os,copy
import math
import tqdm
from scipy.stats import nbinom
import pickle as pk
import os
os.environ["CUDA_VISIBLE_DEVICES"]='2'
# Parameters
torch.manual_seed(0)
device = 'cpu'
#torch.device('cuda') 
A = np.load('/Users/pipipu/Desktop/STZINB/ny_data_full_5min/adj_rand0.npy') # change the loading folder
X = np.load('/Users/pipipu/Desktop/STZINB/ny_data_full_5min/cta_samp_rand0.npy')

In [3]:
num_timesteps_output = 4 
num_timesteps_input = num_timesteps_output

space_dim = X.shape[1]
batch_size = 4
hidden_dim_s = 70
hidden_dim_t = 7
rank_s = 20
rank_t = 4
n_gaussian = 2

epochs = 50 #500

# Initial networks
TCN1 = B_TCN(space_dim, hidden_dim_t, kernel_size=3).to(device=device)
TCN2 = B_TCN(hidden_dim_t, rank_t, kernel_size = 3, activation = 'linear').to(device=device)
TCN3 = B_TCN(rank_t, hidden_dim_t, kernel_size= 3).to(device=device)
TNB = MDN(hidden_dim_t,space_dim,n_gaussian).to(device=device)
#NBNorm_ZeroInflated(hidden_dim_t,space_dim).to(device=device)
SCN1 = D_GCN(num_timesteps_input, hidden_dim_s, 3).to(device=device)
SCN2 = D_GCN(hidden_dim_s, rank_s, 2, activation = 'linear').to(device=device)
SCN3 = D_GCN(rank_s, hidden_dim_s, 2).to(device=device)
SNB = MDN(hidden_dim_s,num_timesteps_output,n_gaussian).to(device=device)
#NBNorm_ZeroInflated(hidden_dim_s,num_timesteps_output).to(device=device)
STmodel = ST_MDN(SCN1, SCN2, SCN3, TCN1, TCN2, TCN3, SNB,TNB).to(device=device)
#ST_NB_ZeroInflated(SCN1, SCN2, SCN3, TCN1, TCN2, TCN3, SNB,TNB).to(device=device)

In [4]:
X = X.T
X = X.astype(np.float32)

X = X.reshape((X.shape[0],1,X.shape[1]))
X = X[:,:,:100]
split_line1 = int(X.shape[2] * 0.6)
split_line2 = int(X.shape[2] * 0.7)
print(X.shape,A.shape)

# normalization
max_value = np.max(X[:, :, :split_line1])

train_original_data = X[:, :, :split_line1]
val_original_data = X[:, :, split_line1:split_line2]
test_original_data = X[:, :, split_line2:]
training_input, training_target = generate_dataset(train_original_data,
                                                    num_timesteps_input=num_timesteps_input,
                                                    num_timesteps_output=num_timesteps_output)
val_input, val_target = generate_dataset(val_original_data,
                                            num_timesteps_input=num_timesteps_input,
                                            num_timesteps_output=num_timesteps_output)
test_input, test_target = generate_dataset(test_original_data,
                                            num_timesteps_input=num_timesteps_input,
                                            num_timesteps_output=num_timesteps_output)
print('input shape: ',training_input.shape,val_input.shape,test_input.shape)

(4489, 1, 100) (4489, 4489)
input shape:  torch.Size([53, 4489, 4, 1]) torch.Size([3, 4489, 4, 1]) torch.Size([23, 4489, 4, 1])


In [5]:
A_wave = get_normalized_adj(A)
A_q = torch.from_numpy((calculate_random_walk_matrix(A_wave).T).astype('float32'))
A_h = torch.from_numpy((calculate_random_walk_matrix(A_wave.T).T).astype('float32'))
A_q = A_q.to(device=device)
A_h = A_h.to(device=device)
# Define the training process
# criterion = nn.MSELoss()
optimizer = optim.Adam(STmodel.parameters(), lr=1e-2)
training_nll   = []
validation_nll = []
validation_mae = []

In [6]:
for epoch in range(epochs):

    ## Step 1, training
    """
    # Begin training, similar training procedure from STGCN
    Trains one epoch with the given data.
    :param training_input: Training inputs of shape (num_samples, num_nodes,
    num_timesteps_train, num_features).
    :param training_target: Training targets of shape (num_samples, num_nodes,
    num_timesteps_predict).
    :param batch_size: Batch size to use during training.
    """
    permutation = torch.randperm(training_input.shape[0])
    epoch_training_losses = []
    for i in range(0, training_input.shape[0], batch_size):
        STmodel.train()
        optimizer.zero_grad()

        indices = permutation[i:i + batch_size]
        X_batch, y_batch = training_input[indices], training_target[indices]
        X_batch = X_batch.to(device=device)
        y_batch = y_batch.to(device=device)
    
        pi_train,pi_g_train,mu_g_train,sigma_g_train = STmodel(X_batch,A_q,A_h)
        #n_train,p_train,pi_train = STmodel(X_batch,A_q,A_h)


        loss = nb_MDN_nll_loss(y_batch,pi_train, pi_g_train,mu_g_train,sigma_g_train)
        
        #loss = nb_zeroinflated_nll_loss(y_batch,n_train,p_train,pi_train)
        loss.backward()
        optimizer.step()
        epoch_training_losses.append(loss.detach().cpu().numpy())
    training_nll.append(sum(epoch_training_losses)/len(epoch_training_losses))
    print(sum(epoch_training_losses)/len(epoch_training_losses))

/Users/pipipu/miniconda3/envs/DLML/lib/python3.9/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


19.100705010550364
19.010907581874303
18.829890523638046
18.785037721906388
18.767018726893834
18.761702946254186
18.75909764426095
18.757761682782853
18.755827358790807
18.758252825055802
18.755607060023717
18.749221801757812
18.748316083635604
18.75291783469064
18.755890165056503
18.75613921029227
18.755666732788086
18.750990867614746
18.74681200299944
18.75399044581822
18.753967557634628
18.752950259617396
18.7479122706822
18.750214304242814
18.752748216901505
18.74919182913644
18.751228060041154
18.748877797807967
18.745121955871582
18.746251787458146
18.744773592267716
18.751914296831405
18.753116880144393
18.752203941345215
18.745682307652064
18.751022611345565
18.751975331987655
18.745358330862864
18.7504916872297
18.751978874206543
18.750537736075266
18.752279145377024
18.75174249921526
18.750738961356028
18.75005681174142
18.74631827218192
18.751503671918595
18.75201920100621
18.748608589172363
18.745958873203822


In [13]:
for epoch in range(epochs):


    with torch.no_grad():
        STmodel.eval()
        val_input = val_input.to(device=device)
        val_target = val_target.to(device=device)

        pi_val,pi_g_val,mu_g_val,sigma_g_val = STmodel(val_input,A_q,A_h)
        #n_val,p_val,pi_val = STmodel(val_input,A_q,A_h)
        #print('Pi_val,mean,min,max',torch.mean(pi_val),torch.min(pi_val),torch.max(pi_val))

        val_loss = nb_MDN_nll_loss(val_target,pi_val, pi_g_val,mu_g_val,sigma_g_val)
        #val_loss    = nb_zeroinflated_nll_loss(val_target,n_val,p_val,pi_val).to(device="cpu")
        validation_nll.append(np.asscalar(val_loss.detach().numpy()))
       
        # Calculate the expectation value
        
        
        val_pred = torch.abs(1-pi_val) *  torch.sum((pi_g_val * mu_g_val),dim = -1)
        print(torch.sum((pi_g_val * mu_g_val),dim = -1))
        #val_pred = (1-pi_val.detach().cpu().numpy())*(n_val.detach().cpu().numpy()/p_val.detach().cpu().numpy()-n_val.detach().cpu().numpy()) # pipred
        #print(val_pred.mean(),pi_val.detach().cpu().numpy().min())
        mae = torch.mean(torch.abs(val_pred - val_target.detach().cpu().numpy()))
        
        #validation_mae.append(mae)

/var/folders/31/4b0w_54n0lg5yfw096pph8bc0000gn/T/ipykernel_80806/1326070190.py:15: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  validation_nll.append(np.asscalar(val_loss.detach().numpy()))


tensor(0.7925)
tensor(0.2075)
tensor(0.7925)
tensor(0.2075)
tensor(0.7925)
tensor(0.2075)
tensor(0.7925)
tensor(0.2075)
tensor(0.7925)
tensor(0.2075)


KeyboardInterrupt: 